In [1]:
import pandas as pd
import matplotlib as mpl 
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv('/Users/sm/Documents/2020-여름/Big Data Contest/jeju_data_ver1/201901-202003.csv')

In [3]:
df.rename(columns = {'REG_YYMM' : 'DATE', 'CARD_SIDO_NM' :'SIDO', 
                     'STD_CLSS_NM' : 'STD', 'SEX_CTGO_CD':'SEX'}, inplace = True)


drop_columns = ['CARD_CCG_NM', 'HOM_SIDO_NM', 'HOM_CCG_NM']
df.drop(drop_columns, axis=1, inplace = True)

#AMT_mean column 추가 (이상치 제거하기 위해서.)
df['AMT_mean'] = np.round(df['AMT'] / df['CNT'], 3)

In [4]:
#이상치 제거하는 코드 업데이트!

def no_outliers(df):
    Q1 = df['AMT_mean'].quantile(0.25)
    Q3 = df['AMT_mean'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    amt_mean_outlier_indices = df[(df['AMT_mean'] < lower_bound) | (df['AMT_mean'] > upper_bound)].index
    
    Q11 = df['CNT'].quantile(0.25)
    Q33 = df['CNT'].quantile(0.75)
    IQRR = Q33 - Q11
    lowerbound = Q11 - 1.5 * IQRR
    upperbound = Q33 + 1.5 * IQRR
    cnt_outlier_indices = df[(df['CNT'] < lowerbound) | (df['CNT'] > upperbound)].index
    
    total_outlier_indices = sorted(list(set(amt_mean_outlier_indices.tolist() + cnt_outlier_indices.tolist())))
    df_out = df.drop(total_outlier_indices, axis = 0, inplace = False)
    
    return df_out

In [5]:
print(df.shape)
df = no_outliers(df)
print(df.shape)

(24697792, 10)
(19715691, 10)


In [6]:
#위에서 만든 열 'AMT_mean'은 이상치를 제거하기 위해 만들었던 열이므로 다시 지운다. 
#뒤에서 groupby를 수행하면서 값에 오차가 커지기 때문 (np.round 이용했으므로)
df.drop(['AMT_mean'], axis = 1, inplace = True)

df = df.groupby(['DATE', 'SIDO', 'STD', 'AGE', 'SEX', 'FLC'])['CSTMR_CNT','AMT','CNT'].sum().reset_index()

#revisit column 추가
df['revisit'] = np.round(1 - df['CSTMR_CNT'] / df['CNT'], 5)

df.drop(['CSTMR_CNT'], axis = 1, inplace = True)

#revisit < 0 : 취소한 고객이 재방문한 고객보다 많음.
#revisit값이 크면 클수록 재방문한 고객수가 많음.

# target 로그 변환
df['AMT'] = np.log1p(df['AMT'])

# one-hot-encoding을 위해 str으로 바꿈
cols = ['SEX', 'FLC']
for col in cols:
    df[col] = df[col].apply(str)
    
df_ohe = pd.get_dummies(df)

<ipython-input-6-55ab883aab99>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = df.groupby(['DATE', 'SIDO', 'STD', 'AGE', 'SEX', 'FLC'])['CSTMR_CNT','AMT','CNT'].sum().reset_index()


In [7]:
print(df.shape)
print(df_ohe.shape)

(190116, 9)
(190116, 76)


In [8]:
date_list = df['DATE'].unique().tolist()

for idx, date in enumerate(date_list):
    globals()['df{}'.format(idx + 1)] = df_ohe[df_ohe['DATE'] == date ]
    
for i in range(len(date_list)):
    drop_column = ['DATE']
    globals()['df{}'.format(i + 1)].drop(drop_column, axis = 1, inplace = True)

/Users/sm/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


일부러 보기 쉽게 df1 ~ df15로 함 (월도 맞출 겸)

201901 data => df1

201902 data => df2

...

202003 data => df15

In [9]:
df1.head()

,AMT,CNT,revisit,SIDO_강원,SIDO_경기,SIDO_경남,SIDO_경북,SIDO_광주,SIDO_대구,SIDO_대전,...,AGE_50s,AGE_60s,AGE_70s,SEX_1,SEX_2,FLC_1,FLC_2,FLC_3,FLC_4,FLC_5
0,10.440010,6,0.16667,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,13.720431,23,0.08696,1,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,14.525909,31,0.09677,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,11.664796,5,0.00000,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,12.577984,18,-0.05556,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [10]:
for i in range(15):
    print(globals()['df{}'.format(i + 1)].values.shape)

(12629, 75)
(12587, 75)
(12704, 75)
(12671, 75)
(12829, 75)
(12760, 75)
(12848, 75)
(12957, 75)
(12792, 75)
(12848, 75)
(12682, 75)
(12820, 75)
(12752, 75)
(12320, 75)
(11917, 75)


월별 데이터의 개수가 다름. 

rnn input은 모두 shape이 같아야 한다.

월별 데이터의 수를 무작위로 10000개 샘플링해서 training하고, 나머지 1800개 정도를 test data로 이용.
(가장 적은 데이터 수가 11917개여서 임의로 1800개로 잡음)

아니면 이상치 제거하지 않고 해도 됨. 둘 다 해보고 괜찮은 걸로 하는걸로.

In [11]:
import random

In [20]:
def get_train_test_data(df):
    num_of_train_data = 10000
    list = []
    rand_num = random.randrange(df.shape[0])
    for i in range(num_of_train_data):
        while rand_num in list:
            rand_num = random.randrange(df.shape[0])
        list.append(rand_num)
        
    train_index_list = sorted(list)
    train_data = df.iloc[train_index_list,:]
    
    total_list = [x for x in range(df.shape[0])]
    index_list = [i for i in total_list if i not in train_index_list]
    
    test_index_list = sorted(random.sample(index_list, 1800))
    test_data = df.iloc[test_index_list,:]
    
    return train_data, test_data

In [21]:
for i in range(len(date_list)):
    globals()['df{}_train'.format(i+1)], globals()['df{}_test'.format(i+1)] = get_train_test_data(globals()['df{}'.format(i+1)])
    print('train_{} shape: '.format(i+1), globals()['df{}_train'.format(i+1)].shape)
    print('test_{} shape: '.format(i+1), globals()['df{}_test'.format(i+1)].shape)

train_1 shape:  (10000, 75)
test_1 shape:  (1800, 75)
train_2 shape:  (10000, 75)
test_2 shape:  (1800, 75)
train_3 shape:  (10000, 75)
test_3 shape:  (1800, 75)
train_4 shape:  (10000, 75)
test_4 shape:  (1800, 75)
train_5 shape:  (10000, 75)
test_5 shape:  (1800, 75)
train_6 shape:  (10000, 75)
test_6 shape:  (1800, 75)
train_7 shape:  (10000, 75)
test_7 shape:  (1800, 75)
train_8 shape:  (10000, 75)
test_8 shape:  (1800, 75)
train_9 shape:  (10000, 75)
test_9 shape:  (1800, 75)
train_10 shape:  (10000, 75)
test_10 shape:  (1800, 75)
train_11 shape:  (10000, 75)
test_11 shape:  (1800, 75)
train_12 shape:  (10000, 75)
test_12 shape:  (1800, 75)
train_13 shape:  (10000, 75)
test_13 shape:  (1800, 75)
train_14 shape:  (10000, 75)
test_14 shape:  (1800, 75)
train_15 shape:  (10000, 75)
test_15 shape:  (1800, 75)


In [22]:
#나중에 train과 test로 나눈 후에 만들기.

for i in range(len(date_list)):
    globals()['train_target_{}'.format(i+1)] = globals()['df{}_train'.format(i + 1)]['AMT']
    globals()['train_features_{}'.format(i+1)] = globals()['df{}_train'.format(i + 1)].drop(['AMT'], axis = 1, inplace = False)
    
    globals()['test_target_{}'.format(i+1)] = globals()['df{}_test'.format(i + 1)]['AMT']
    globals()['test_features_{}'.format(i+1)] = globals()['df{}_test'.format(i + 1)].drop(['AMT'], axis = 1, inplace = False)
    

In [23]:
def reshaping_train_features(df):
    return df.values.reshape(1,10000,74)

train_input = reshaping_train_features(train_features_1)
for i in range(len(date_list)-1):
    train_input = np.concatenate((train_input,reshaping_train_features(globals()['train_features_{}'.format(i+2)])))

RNN에서 시계열 데이터는 총 15개.

각 시계열 데이터는 10000개의 데이터를 가지고 74개의 feature를 지니고 있음.

각 시계열 데이터에서의 원하는 타겟은 단 하나(AMT). 

In [24]:
train_input.shape

(15, 10000, 74)

In [25]:
def reshaping_train_targets(df):
    return df.values.reshape(1,-1,1)

train_output = reshaping_train_targets(train_target_1)
for i in range(len(date_list)-1):
    train_output = np.concatenate((train_output, reshaping_train_targets(globals()['train_target_{}'.format(i+2)])))

In [26]:
train_output.shape

(15, 10000, 1)

In [27]:
def reshaping_test_features(df):
    return df.values.reshape(1,1800,74)

test_input = reshaping_test_features(test_features_1)
for i in range(len(date_list)-1):
    test_input = np.concatenate((test_input,reshaping_test_features(globals()['test_features_{}'.format(i+2)])))

In [28]:
test_input.shape

(15, 1800, 74)

In [29]:
def reshaping_test_targets(df):
    return df.values.reshape(1,-1,1)

test_output = reshaping_test_targets(test_target_1)
for i in range(len(date_list)-1):
    test_output = np.concatenate((test_output, reshaping_test_targets(globals()['test_target_{}'.format(i+2)])))

In [30]:
test_output.shape

(15, 1800, 1)